<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/Groceries/Lotus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU scrapy selenium

In [8]:
%%writefile Lotus.py
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import scrapy
from scrapy.crawler import CrawlerProcess

class LotusSpider(scrapy.Spider):
    name = 'Lotus'

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        options = webdriver.chrome.options.Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--enable-javascript")
        options.add_argument("--enable-cookies")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-web-security")
        options.add_argument("--incognito")
        self.driver = webdriver.Chrome(options=options)

    def start_requests(self):
        urls = [
            "https://www.lotuss.com.my/en/category/grocery/cereals/cereals"
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_links)

    def parse_links(self, response):
        self.driver.get(response.url)

        elem_present = ""
        while not elem_present:
            try:
                elem_present = self.driver.find_element(By.XPATH, """
                    //div[@class='product-grid-item']
                """)
            except:
                continue

        selector = scrapy.Selector(text=self.driver.page_source)
        category_links = selector.css("div.carousel a")
        if len(category_links)!=0:
            yield from response.follow_all(category_links,
                callback=self.parse_links)
        else:
            last_height = self.driver.execute_script(
                "return document.body.scrollHeight"
            )

            while True:
                self.driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight)"
                )
                time.sleep(3)
                new_height = self.driver.execute_script(
                    "return document.body.scrollHeight"
                )
                if new_height==last_height:
                    break
                last_height = new_height

            selector = scrapy.Selector(text=self.driver.page_source)
            product_links = selector.css("""
                div#product-list a
            """)

            yield from response.follow_all(product_links,
                callback=self.parse_items)

    def parse_items(self, response):
        print(response.url)
    #     self.driver.get(response.url)

    #     elem_present = ""
    #     while not elem_present:
    #         try:
    #             elem_present = self.driver.find_element(By.XPATH, """
    #                 //div[@class='product-grid-item']
    #             """)
    #         except:
    #             continue

    #     selector = scrapy.Selector(text=self.driver.page_source)
    #     product_links = selector.css("""
    #         div.product-grid-item
    #     """)
    #     print(product_links)

process = CrawlerProcess()
process.crawl(LotusSpider)
process.start()

Overwriting Lotus.py


In [9]:
%%bash
python Lotus.py

202
https://www.lotuss.com.my/en/product/quaker-instant-oatmeal-135kg-4058097
https://www.lotuss.com.my/en/product/lotuss-rolled-oats-800g-74691325
https://www.lotuss.com.my/en/product/lotuss-cereal-flakes-500g-foc-50g-74691295
https://www.lotuss.com.my/en/product/nestle-school-pack-6s-140g-1283839
https://www.lotuss.com.my/en/product/quaker-q-vital-3-in-1-van-15-x-28g-73468932
https://www.lotuss.com.my/en/product/quaker-q-vital-3-in-1-ori-15-x-28g-73468800
https://www.lotuss.com.my/en/product/75059932
https://www.lotuss.com.my/en/product/quaker-whole-rolled-oats-800g-72729546
https://www.lotuss.com.my/en/product/quaker-instant-oatmeal-1kg-foil-70941149
https://www.lotuss.com.my/en/product/nestle-koko-krunch-multipack-6x25g-126179
https://www.lotuss.com.my/en/product/quaker-quick-cook-oatmeal-1kg-foil-70941157
https://www.lotuss.com.my/en/product/yogood-choc+nut-muesli-bar-138g-6s-4554892
https://www.lotuss.com.my/en/product/75059959
https://www.lotuss.com.my/en/product/nestle-koko-kru

2024-08-12 14:01:05 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-08-12 14:01:05 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.10.12 (main, Jul 29 2024, 16:56:48) [GCC 11.4.0], pyOpenSSL 24.2.1 (OpenSSL 3.2.2 4 Jun 2024), cryptography 42.0.8, Platform Linux-6.1.85+-x86_64-with-glibc2.35
2024-08-12 14:01:05 [selenium.webdriver.common.selenium_manager] DEBUG: Selenium Manager binary found at: /usr/local/lib/python3.10/dist-packages/selenium/webdriver/common/linux/selenium-manager
2024-08-12 14:01:05 [selenium.webdriver.common.selenium_manager] DEBUG: Executing process: /usr/local/lib/python3.10/dist-packages/selenium/webdriver/common/linux/selenium-manager --browser chrome --language-binding python --output json
2024-08-12 14:01:05 [selenium.webdriver.common.selenium_manager] DEBUG: Driver path: /root/.cache/selenium/chromedriver/linux64/127.0.6533.99/chromedriver
2024-08-12 14: